<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/use_cases/retail_store_analysis_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit
!npm install localtunnel

!wget https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/pop_rad_data.parquet -O pop_rad_data.parquet
!wget https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/poi_accessibility.parquet -O poi_accessibility.parquet

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 1.044s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
--2024-05-07 05:06:34--  https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/pop_rad_data.parquet
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kavyajeetbora/modern_geospatial_stack/master/use_cases/pop_rad_data.parquet [following]
--2024-05-07 05:06:34--  https:

In [27]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.75.45.42


In [37]:
%%writefile app.py
import streamlit as st
import pydeck as pdk
import pandas as pd
from matplotlib import colormaps

## STREAMLIT CONFIGURATION
## --------------------------------------------------------------------------------##
st.set_page_config(page_title='Retail Store Location Analysis', page_icon='', layout="wide")

pop_rad_data = pd.read_parquet('pop_rad_data.parquet')
poi_accessibility = pd.read_parquet('poi_accessibility.parquet')

def color_code_value(value, cmap):
    color = cmap(value)
    scaled_colors = list(map(lambda x: int(x * 255), color[:3]))
    scaled_alpha = int(color[3] * 100)

    scaled_colors += [scaled_alpha]
    return scaled_colors


def colormap_dataframe(df, value_col, cmap):

    xdf = df.copy()

    ## Scale the temperature values between 0-1; using MinMax Scaler
    xdf[f"norm_{value_col}"] = (xdf[value_col] - xdf[value_col].min()) / (
        xdf[value_col].max() - xdf[value_col].min()
    )
    xdf["color"] = xdf[f"norm_{value_col}"].apply(lambda x: color_code_value(x, cmap))
    xdf[["R", "G", "B", "A"]] = pd.DataFrame(xdf["color"].to_list())
    xdf = xdf.drop(["color", f"norm_{value_col}"], axis=1)

    ## formatting the temperature value upto 1 decimal place
    xdf[value_col] = xdf[value_col].round(1)

    return xdf


def create_h3_hex_layer(df, hex_col):

    # Define a layer to display on a map
    layer = pdk.Layer(
        "H3HexagonLayer",
        df,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon=hex_col,
        get_fill_color="[R, G, B, A]",
    )

    return layer

hex_col = 'hex8'

## Radiance data
radiance = colormap_dataframe(pop_rad_data[['hex8', 'radiance']], value_col='radiance', cmap=colormaps['cividis'])
rad_layer = create_h3_hex_layer(radiance, hex_col)

# Population data
pop_age_sex = colormap_dataframe(pop_rad_data[['hex8', 'population', 'sum_male', 'sum_female']], value_col='population', cmap=colormaps['seismic'])
pop_age_sex_layer = create_h3_hex_layer(pop_age_sex, hex_col)

# Density of Amenity
poi_density = colormap_dataframe(poi_accessibility[['hex8', 'poi_count']], value_col='poi_count', cmap=colormaps['viridis'])
poi_density_layer = create_h3_hex_layer(poi_density, hex_col)

# Accessibility
accessibility = colormap_dataframe(poi_accessibility[['hex8', 'time_min']], value_col='time_min', cmap=colormaps['YlOrRd'])
accessibility_layer = create_h3_hex_layer(accessibility, hex_col)

def generate_tooltip(text):
    tooltip = {
            "html": text,
            "style": {
                "backgroundColor": "#4CAF50",   # Green shade for background
                "color": "#FFFFFF",             # White for text color
                "border": "2px solid #4CAF50",  # Matching border color
                "borderRadius": "5px",          # Rounded corners
                "boxShadow": "2px 2px 10px rgba(0, 0, 0, 0.2)"  # Soft shadow effect
            }
        }

    return tooltip

options = {
   "Density of Amenities": (poi_density_layer, generate_tooltip("Amenities Count: {poi_count}")),
    "Public Transport Accessibility": (accessibility_layer, generate_tooltip("Pedestrian Accessibility: {time_min} mins")),
    "Population Density": (pop_age_sex_layer, generate_tooltip('''Population: {population} <br>
        Male: {sum_male} % <br>
        Female: {sum_female} %
        ''')),
    "Nighttime Lights": (rad_layer, generate_tooltip("Radiance: {radiance}"))
}

def create_deck(hex_layer, tooltip):
    x, y = (77.06463665845695, 28.437398083125167)
    view_state = pdk.ViewState(latitude=y, longitude=x, zoom=12, bearing=0, pitch=15)
    # Render
    r = pdk.Deck(layers=[hex_layer], initial_view_state=view_state, tooltip=tooltip)

    return r


st.sidebar.markdown("## Map Controls")

parameter = st.sidebar.selectbox(
    label="Parameter",
    options=options.keys(),
)

hex_layer, tooltip = options[parameter]
deck = create_deck(hex_layer, tooltip)

st.pydeck_chart(deck, use_container_width=True)

Overwriting app.py


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 3.106s
your url is: https://sad-otters-slide.loca.lt
